# Simulating Molecules using Orbital Optimization VQE

The Oribital Optimization VQE is a method based on the traditional VQE, where the Hamiltonian basis is classically optimized similarly to the variational parameters of the circuit ansatz.
We start by shortly summarizing VQE to facilitate the understanding of Oribital Optimization VQE.

<img src="aux_files/oovqe_h4_orbitals.png" width="300">

##### Figure: Illustration of the change in the shape of the molecular orbitals of  H$_4$ after converging Orbital Optimization VQE, this calculation performed later in this notebook.

Reference: "Quantum orbital-optimized unitary coupled cluster methods in the strongly correlated regime: Can quantum algorithms outperform their classical equivalents?", J. Chem. Phys. 152, 124107 (2020); https://doi.org/10.1063/1.5141835 

### VQE <v id='vqe'></v>

In 2014, Peruzzo *et al.* proposed VQE to estimate the ground state energy of a molecule using much shallower circuits. 
Given a Hermitian operator $\hat H$ with an unknown minimum eigenvalue $\lambda_{min}$, associated with the eigenstate $|\psi_{min}\rangle$, VQE provides an estimate $\lambda_{\theta}$ bounded from below by $\lambda_{min}$:

\begin{align*}
    \lambda_{min} \le \lambda_{\theta} \equiv \langle \psi(\theta) |\hat H|\psi(\theta) \rangle
\end{align*}  

where $|\psi(\theta)\rangle$ is the eigenstate associated with $\lambda_{\theta}$. By applying a parameterized circuit, represented by $U(\theta)$, to some arbitrary starting state $|\psi\rangle$, the algorithm obtains an estimate $U(\theta)|\psi\rangle \equiv |\psi(\theta)\rangle$ on $|\psi_{min}\rangle$. The estimate is iteratively optimized by a classical optimizer changing the parameter $\theta$ minimizing the expectation value of $\langle \psi(\theta) |\hat H|\psi(\theta) \rangle$. 

##### Reference for additional details:
Peruzzo, A., McClean, J., Shadbolt, P. et al. A variational eigenvalue solver on a photonic quantum processor. Nat Commun 5, 4213 (2014). https://doi.org/10.1038/ncomms5213

### Orbital Optimization VQE <ov id='oovqe'></ov>

Unlike VQE, we perform a basis transformation of the Hamiltonian, which requires introducing additonal variational parameters (e.g. $\kappa$) to control the basis rotations. 
In this way, the overlap between the states generated by the given wavefunction ansatz and the ground state is maximized.

Therefore, differently from VQE, the energy estimate $\langle \psi(\theta) | \hat H'(\kappa)|\psi(\theta) \rangle$ is minimized by a classical optimizer, changing the parameters $\theta$ and also $\kappa$.
The converged energy in this approach is generally lower than the one of VQE

\begin{align*}
    \lambda_{min} \le \text{min}_{\theta, \kappa}\langle \psi(\theta) |\hat H'(\kappa)|\psi(\theta) \rangle \le \text{min}_{\theta} \langle \psi(\theta) |\hat H|\psi(\theta) \rangle
\end{align*}  

The optimization of $\kappa$ parameters can be done simultaneously with the $\theta$ parameters of the wavefunction ansatz or iteratively (i.e. separate optimizations of $\theta$ and $\kappa$) until convergence.
This implies that the Hamiltonian coefficients have to be recomputed as soon as $\kappa$ change.

### Details of Orbital Optimization VQE <ov id='oovqe_details'></ov>

To show how the molecular Hamiltonian is transformed, we start with its definition

$$
\begin{aligned}
\hat{H} &=\sum_{r s} h_{r s} \hat{a}_{r}^{\dagger} \hat{a}_{s} \\
&+\frac{1}{2} \sum_{p q r s} g_{p q r s} \hat{a}_{p}^{\dagger} \hat{a}_{q}^{\dagger} \hat{a}_{r} \hat{a}_{s}+E_{N N}
\end{aligned}
$$

where the $h_{r s}$  and $g_{p q r s}$ are the one-/two-body integrals and $E_{N N}$ the nuclear repulsion energy.
A suitable unitary transformation (e.g. $O(\kappa)= \exp{(\hat\kappa)}$) can be applied to the Hamiltonian so that the spectrum remains unchanged.
More specifically, the Hamiltonian is similaity transformed (orbital rotated) in the following way

\begin{equation}
\hat H \rightarrow \hat H' = \hat H'(\kappa)
= \hat{O}^{-1} \hat{H} \hat{O} = \exp{(-\hat\kappa)} \hat{H} \exp{(\hat\kappa)}
\end{equation}

where we introduced the anti-Hermitian operator composed of single excitations

\begin{equation}
\hat\kappa = \sum_{rs} \kappa_{rs} \hat{a}^{\dagger}_r \hat{a}_s
\end{equation}

This entails that the unitary transformation $\exp{(\hat\kappa)}$ brings additional cost in terms of quantum gates (i.e. by mapping it to a quantum circuit).
However, we can include this transformation in the integrals on the classical computer, avoiding introducing any quantum gates.

The action of this transformation on the creation and annihilation operators, in the second quantization, is

\begin{align}
    \tilde{\hat{a}}_r &=\exp{(-\hat\kappa)} \hat{a}_r \exp{(\hat\kappa)} = \sum_{s} R^*_{sr} {\hat{a}}_s \\
    \tilde{\hat{a}}^{\dagger}_r &=\exp{(-\hat\kappa)} \hat{a}^{\dagger}_r \exp{(\hat\kappa)} = \sum_{s} R_{sr} {\hat{a}}^\dagger_s 
\end{align}

which corresponds to the change in the spin orbitals $\phi_{r}$

\begin{equation}
\phi_{r}=\sum_{s} R_{rs} \, \phi_s
\end{equation}

and therefore, the orbital rotation can be applied to one-/two-electron integrals by means of a basis change 

\begin{align}
\tilde{\hat{h}}_{rs} &= \sum_{ab}C^{\alpha, *}_{ar} C^{\alpha}_{bs} h_{rs}  \\
\tilde{\hat{g}}_{pqrs} &= \sum_{abcd}C^{\alpha, *}_{ap} C^{\alpha, *}_{bq} C^{\alpha}_{cr} C^{\alpha}_{ds} g_{pqrs}
\end{align}

where $a,b,c,d$ label the atomic orbitals (AOs) and $p,q,r,s$ the molecular orbitals (MOs).
The $C^{\alpha}$ and $C^{\beta}$ correspond to AO-to-MO coefficients that serve to linearly combine the AOs into MOs (including the spin components $\alpha, \beta$). 
The orbital rotation is therefore performed by acting on original AO-to-MO coefficient matrices $C^{\alpha}_{\text{RHF}}$ and $C^{\beta}_{\text{RHF}}$ obtained with a restricted Hartree-Fock calculation ($C^{\alpha}_{\text{RHF}} = C^{\beta}_{\text{RHF}}$) on a classical computer as

\begin{align}
C^{\alpha}&={C}_{\text{RHF}}^{\alpha} \exp{(-\kappa_{\alpha})}  \\
C^{\beta}&={C}_{\text{RHF}}^{\beta} \exp{(-\kappa_{\beta})}
\end{align}

This transformation preserves orthonormality of the orbitals.
In addition, the RHF orbtials are real and the spin-restriction forces $\kappa = \kappa_{\alpha} = \kappa_{\beta}$ which implies $C^{\alpha}=C^{\beta}$. 
As described above, the elements of the anti-hermitian $\kappa$ matrix control the orbital rotations.
In this matrix, there are $N_{MO}(N_{MO}-1)/2$ independent elements, that directly become the variational parameters of the algorithm (i.e. in addition to circuit variational parameters).

We can therefore write the Hamiltonian in the rotated orbital basis as

\begin{align}
{\hat{H}'} =\hat H'(\kappa) = &\sum_{rs} \tilde{\hat{h}}_{rs} \,  {\hat{a}}^{\dagger}_r {\hat{a}}_s  \\ &+
\frac{1}{2}\sum_{rstu}\tilde{\hat{g}}_{rstu} \,   {\hat{a}}^{\dagger}_r {\hat{a}}^{\dagger}_s {\hat{a}}_u {\hat{a}}_t + E_{NN} 
\end{align}

##### Reference for additional details:

See Section III, Methods, of "Quantum orbital-optimized unitary coupled cluster methods in the strongly correlated regime: Can quantum algorithms outperform their classical equivalents?", J. Chem. Phys. 152, 124107 (2020); https://doi.org/10.1063/1.5141835 




### Necessary components

In [1]:
from qiskit.aqua import QuantumInstance
from qiskit.aqua import AquaError
from qiskit.aqua.algorithms import VQE, NumPyMinimumEigensolver
from qiskit.aqua.components.optimizers import COBYLA

from qiskit.chemistry.algorithms.ground_state_solvers import OrbitalOptimizationVQE, GroundStateEigensolver
from qiskit.chemistry.algorithms.ground_state_solvers.minimum_eigensolver_factories.\
            vqe_uccsd_factory import VQEUCCSDFactory
from qiskit.chemistry.transformations.fermionic_transformation import FermionicTransformation, FermionicQubitMappingType
from qiskit.chemistry.components.initial_states import HartreeFock
from qiskit.chemistry.components.variational_forms import UCCSD
from qiskit.chemistry.drivers import PySCFDriver, UnitsType, Molecule
from qiskit.chemistry.core import TransformationType, QubitMappingType

from qiskit.providers.basicaer import BasicAer

### H$_2$ molecule: Orbital Optimization VQE 

First, we perform the restricted HF calculation to compute the Hamiltonian coefficients using `PySCFDriver`

In [2]:
molecule = Molecule(geometry=[['H', [.0, .0, .0]],
                              ['H', [.0, .0, .76]]],
                     charge=0, multiplicity=1)
driver = PySCFDriver(molecule = molecule, unit=UnitsType.ANGSTROM, basis='sto3g')

Then, we can select the `initial_point` for the algorithm (assuming one knows the number of parameters).
The initial point contains the parameters for the $\theta$ and then $\kappa$ (matrix elements flattened into a vector) in this order. 
When None is provided, the initial point elements are set to 0.1.

In [3]:
initial_point = None # also can be supplied as a list [...]

We can define the classical optimizer that will change the parameters $\kappa$ (orbitals) and $\theta$ (circuit gate rotation) of the algorithm

In [4]:
optimizer = COBYLA(maxiter=1000) 

We have to define how the second quantized fermionic operators are mapped to quantum computer (Pauli strings).
We select for instance the Jordan-Wigner mapping that maps each spin-orbital to a single qubit using the `FermionicTransformation` class

In [5]:
transformation = FermionicTransformation(qubit_mapping=FermionicQubitMappingType.JORDAN_WIGNER, two_qubit_reduction=False)

- Note that a useful transformation is `FermionicQubitMappingType.PARITY` with `two_qubit_reduction=True` which always reduces by 2 the number of required qubits.

Next, we specify on which simulator the algorithm runs. 
We choose the `statevector_simulator` that simulates the algorithm using matrix-vector multiplications 

In [6]:
quantum_instance = QuantumInstance(BasicAer.get_backend('statevector_simulator'))

We can then initialize the solver using the `VQEUCCSDFactory` that constructs in itself the `VQE` class instance with `q-UCCSD` variational form.
- In particular the `q-pUCCD` is chosen here without the double excitations of electrons of the same spin. 'p' stands for 'pair' and 'D' for double. Which is basically an ansatz made from exponentiation of double excitations of electrons that are restricted to be on the same spatial orbital. In the case of H$_2$ in STO-3G basis, there is anyway only 1 double excitation available so `q-pUCCD` = `q-UCCD` but this will not be the case later on when we show the case of H$_4$ molecule.

The pUCCD Ansatz is defined as
$$
|\Psi({\theta})\rangle=e^{\hat{T}_2({\theta})-\hat{T}_2^{\dagger}({\theta})}\left|\Phi_{0}\right\rangle
$$

with $\hat{T}_2({\theta})$ given in this case by 

$$
\hat{T}_{2}({\theta})=\sum_{m i} \theta_{m_{\uparrow} m_{\downarrow} i_{\uparrow} i_{\downarrow}} \hat{a}_{m, \uparrow}^{\dagger} \hat{a}_{m, \downarrow}^{\dagger} \hat{a}_{i, \uparrow} \hat{a}_{i, \downarrow}
$$
where $|\Phi_{0}\rangle$ is the reference state (`initial_state`) and $i,m$ are the spin-orbital labels ($i=$ occupied, $m=$ virtual).
- We do this to shorten the ansatz circuit with a trade-off of losing accuracy. 
- By using this shortened wavefunction ansatz, we will illustrate how Orbital Optimization can improve the results.

<img src="aux_files/oovqe_uccsd_methods.png" width="400">

##### Figure: Possible variants of q-UCCSD circuit ansatz available in Qiskit. They correspond to some subsets of electronic excitations from occupied to virtual spin-orbitals, which are known to provide qualitatively correct potential energy surfaces.

In [7]:
solver = VQEUCCSDFactory(quantum_instance=quantum_instance,
                         optimizer=optimizer, 
                         excitation_type='d', # only use double excitations
                         same_spin_doubles=False, # do not excite electrons in the same-spin manifold
                         method_doubles='pucc') # restrict to paired excitations

Below, we initialize the Orbital Optimization VQE instance that borrows VQE functionnalities for the expectation value calculation and initializes the elements of the $\kappa$ matrix.
- One can select the `iterative_oo` to be True in order to optimize separately $\kappa$ and $\theta$. `iterative_oo_iterations` sets how many sequential optimizations are done in the `iterative_oo` scheme. By default, both are optimized simultaneously.

In [8]:
calc = OrbitalOptimizationVQE(transformation, 
                              solver, 
                              iterative_oo=False,
                              iterative_oo_iterations=2,
                              initial_point=initial_point)

To run the `OrbitalOptimizationVQE`, we simply call the `solve` method that uses the `driver` which contains the Hamiltonian coefficients ($h_{r s}$  and $g_{p q r s}$, see the definitions at the end of the notebook)

Optional: to see the details of the code execution, one can display the logger outputs by running the code below before `algo_result = calc.solve(driver)`:

```import logging
from qiskit.aqua import set_qiskit_aqua_logging
set_qiskit_aqua_logging(logging.INFO)```

In [9]:
algo_result = calc.solve(driver)

At this stage, we perhaps want to know how many additional parameters ($\kappa$) are in the Orbital Optimized VQE in comparison to traditional VQE. 
We can print out

In [10]:
print('total number of parameters:', calc._num_parameters_oovqe)
print('number of parameters for the ansatz (thetas):', calc.var_form_num_parameters)
print('number of parameters for orbital rotations (kappas):', calc._num_parameters_oovqe - calc.var_form_num_parameters)

total number of parameters: 2
number of parameters for the ansatz (thetas): 1
number of parameters for orbital rotations (kappas): 1


One can see all the results by printing `algo_result.eigenenergies` or see individually, the most relevant ones

In [11]:
# with the energy given by
print('energy: ',algo_result.eigenenergies)

# the thetas parameters
print('thetas: ',algo_result.optimal_point_ansatz)

# and the kappas parameters
print('kappas: ', algo_result.optimal_point_orbitals)

energy:  [(-1.8330814019476391+0j)]
thetas:  [-0.11680453]
kappas:  [3.25379601e-05]


### H$_4$ molecule : improvement using Orbital Optimized VQE vs VQE
#### Orbital Optimized VQE run
In the following, we want to demonstrate in the case of a larger system the advantage of using the Orbital Optimized VQE w.r.t VQE. 
We will see that the energy obtained with Orbital Optimized VQE is significantly closer to the exact energy for STO-3G basis set.

Warning: the cell below can take ~10 - 15 min to execute for random `initial_point`, so we start from a good initial point to shorten the simulation time. 

In [12]:
molecule = Molecule(geometry=[['H', [1.738000, .0, .0]],
                              ['H', [.15148, 1.73139, .0]],
                              ['H', [-1.738, .0, .0]],
                              ['H', [-0.15148, -1.73139, .0]]
                             ],
                     charge=0, multiplicity=1)
driver = PySCFDriver(molecule = molecule, unit=UnitsType.ANGSTROM, basis='sto3g')
initial_point = [-2.68841416e-05, -6.62283571e-01, -6.62267824e-01,  6.77157564e-05,
                  7.85427156e-01,  1.33755753e-05,  4.71396034e-05,  3.96299183e-05,
                 -4.91861849e-05,  7.85453343e-01] 
optimizer = COBYLA(maxiter=10, rhobeg=0.01) # rhobeg defines the perturbation of the variational parameters 
transformation = FermionicTransformation(qubit_mapping=FermionicQubitMappingType.JORDAN_WIGNER, two_qubit_reduction=False)
quantum_instance = QuantumInstance(BasicAer.get_backend('statevector_simulator'))
solver = VQEUCCSDFactory(quantum_instance=quantum_instance,
                         optimizer=optimizer, 
                         excitation_type='d', # only use double excitations
                         same_spin_doubles=False, # do not excite electrons in the same-spin manifold
                         method_doubles='pucc') # restrict to paired excitations
calc = OrbitalOptimizationVQE(transformation, 
                              solver, 
                              iterative_oo=False,
                              iterative_oo_iterations=2,
                              initial_point=initial_point)
algo_result_h4 = calc.solve(driver)
print('energy: ', algo_result_h4.eigenenergies)
print('variational parameters: ', algo_result_h4.optimal_point)

energy:  [(-3.0413733078625302+0j)]
variational parameters:  [-2.68841416e-05 -6.62283571e-01 -6.62267824e-01  6.77157564e-05
  7.85427156e-01  1.33755753e-05  4.71396034e-05  3.96299183e-05
 -4.91861849e-05  7.85453343e-01]


The converged energy should be around $-3.041$ Ha.

### H$_4$ molecule: VQE run
We want to obtain the VQE energy to compare with the energy obtained by Orbital Optimization VQE

In [13]:
vqe_solver = VQEUCCSDFactory(QuantumInstance(BasicAer.get_backend('statevector_simulator')))

solver = VQEUCCSDFactory(quantum_instance=quantum_instance,
                         optimizer=optimizer, 
                         excitation_type='d', # only use double excitations
                         same_spin_doubles=False, # do not excite electrons in the same-spin manifold
                         method_doubles='pucc') # restrict to paired excitations
calc = GroundStateEigensolver(transformation, solver)
result_vqe = calc.solve(driver)
print(result_vqe)

=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -2.63341025995
  - computed part:      -2.63341025995
  - frozen energy part: 0.0
  - particle hole part: 0.0
~ Nuclear repulsion energy (Hartree): 1.168125190117
> Total ground state energy (Hartree): -1.465285069834
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 4.000 S: 0.000 S^2: 0.000 M: -0.000
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [0.0  0.0  0.0]
 
  0: 
  * Electronic dipole moment (a.u.): [0.0  0.0  0.0]
    - computed part:      [0.0  0.0  0.0]
    - frozen energy part: [0.0  0.0  0.0]
    - particle hole part: [0.0  0.0  0.0]
  > Dipole moment (a.u.): [0.0  0.0  0.0]  Total: 0.0
                 (debye): [0.0  0.0  0.0]  Total: 0.0
 


- We obtain $-2.779$ Ha as the electronic energy, significantly higher than the one obtained with the Orbital Optimization VQE ($-3.041$ Ha).

- This means that the orbital rotation allowed for the given circuit to reach the states (closer to the ground state) that were previously unreachable.

Now, let's run the full calculation to get the exact energy with `NumPyMinimumEigensolver`

In [14]:
qubit_op, _ = transformation.transform(driver)
calc_exact = NumPyMinimumEigensolver(qubit_op)
result_exact = calc_exact.run()
print(result_exact['eigenvalue'])

(-3.043965328074287+3.1165781158217456e-17j)


Therefore, the Orbital Optimization VQE allowed us to get significantly closer to the ground state energy ($-3.043$ Ha) than the VQE algorithm.

In [15]:
import qiskit.tools.jupyter
%qiskit_version_table
%qiskit_copyright